In [4]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe
import requests
from bs4 import BeautifulSoup
from time import sleep

### スプレッドシートの設定
・共有にクライアントメールアドレスを登録

・権限者は「編集者」とする

In [7]:
#bot対策にブロックされないようにユーザーエージェントを設定
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.69'
headers = {'User-Agent': ua}

#URLを指定（後で末尾にページ番号を追加）
url = 'https://www.homes.co.jp/akiyabank/case-study/'

#検索結果ページ数を取得
res = requests.get(url, headers=headers)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser')
pagenumbers_from_html = soup.select('li.page-item a')
pagenumber = []

for pn in pagenumbers_from_html:
    pn = pn.text
    if (pn != '›'):
        pagenumber.append(pn)
        
pagenumber_max = int(pagenumber[-1])

#全ての検索ページのURLリストを作成
url_base = 'https://www.homes.co.jp/akiyabank/case-study?page='

#情報格納のための空配列を用意
Name = []
Year = []
Summary = []
Type = []

for i in range (1, pagenumber_max+1): 
    print('ページ数:', i)
    
    #アクセス先のURLを作成
    target_url = url_base + str(i)
    
    #取得したデータをresに導入+文字化け防止
    res = requests.get(target_url, headers=headers)
    res.encoding = 'utf-8'
    
    #スクレイピング先のサーバーに負荷をかけない対策
    sleep(1)
    
    soup = BeautifulSoup(res.text, 'html.parser')
    
    bukkenbox_from_html = soup.find_all('div', attrs={'class':'mod-result-bukkenBox'})

    #空配列に情報を格納
    for bb in bukkenbox_from_html:
        n = bb.select_one('h1').text.replace('\t\t\t\t\t\t\t','') #名称
        y = bb.select_one('p').text.replace('実施年度：','').replace('\n','').replace('          ','') #実施年
        s = bb.select_one('dd').text #概要
        t = bb.find_all('dd')[1].text.replace('\n','') #活用タイプ
        Name.append(n)
        Year.append(y)
        Summary.append(s)
        Type.append(t)      
       
    #DFデータ定義
    data_list = {
        '実施事業者' : Name,
        '実施年度' : Year,
        '概要' : Summary,
        '取り組み手法' : Type
    }

    df = pd.DataFrame(data_list)
    

ページ数: 1
ページ数: 2
ページ数: 3
ページ数: 4
ページ数: 5
ページ数: 6
ページ数: 7
ページ数: 8
ページ数: 9
ページ数: 10
ページ数: 11
ページ数: 12
ページ数: 13


In [11]:
df

,実施事業者,実施年度,概要,取り組み手法
0,ＮＰＯ法人都市環境協会,2021,介護施設入所時に空き家の管理責任者指名の啓発\r\n管理責任者と専門家と新潟市が一体となった...,空き家利活用手法の構築
1,COMMONS ISUMI,2021,事務局の運営\r\n空家調査\r\nイベントの開催\r\nツアーの実施\r\nチャレンジショ...,ワークショップの開催、空き家利活用手法の構築、空き家活用イベント...
2,一般財団法人島根県建築住宅センター,2021,新たな空き家活用のモデルとなる、地域と大学が連携したシェアハウスを整備\r\n空き家の継続的...,空き家利活用手法の構築
3,Japan asset management株式会社,2021,事業ブランドの構築\r\nウェブサイトの構築\r\nウェブサイトと事業の告知\r\nモデルル...,ホームページの作成、セミナーの開催、啓発冊子の作成 ...
4,庄原市空き家解決専門家ネットワーク,2021,多拠点居住、コワーキング等社会ニーズに空き家を利活用する取組\r\n空き家バンク物件掲載、移...,セミナーの開催、相談会の実施、ワンストップ相談体制構築、空き家情...
...,...,...,...,...
359,一般社団法人岡山住まいと暮らしの相談センター,2017,空き家所有者向け相談会、セミナーの実施\n笠岡市内諸島部における空き家の現地調査,セミナーの開催、相談会の実施
360,木綿街道再生モデル事業,2017,空き家利活用の先進事例調査\n先進事例調査および地域概況を踏まえた相談会・勉強会の企画\n人...,人材育成研修会の実施
361,入江不動産株式会社,2017,空き家利活用や適正管理に関するパンフレットの作成、配布による周知\n所有者向け説明会・相談会...,ホームページの作成、啓発冊子の作成、相談会の実施 ...
362,ＮＰＯ法人頴娃おこそ会,2017,当該ＮＰＯが借り受け人として無償ベースで行っている契約内容に関する課題の洗い出し\n空き家再...,セミナーの開催、ワークショップの開催、啓発冊子の作成、人材育成研...


In [14]:
#各種情報設定
SP_COPE = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]
SP_CREDENTIAL_FILE = 'my-first-spread-sheet-api-1d4c0fa8908d.json'
SP_SHEET_KEY = '1ZL2U8IaXIwWLs7E_ls3gnNxBWj8YkHW7YltLH66UYWA'
SP_SHEET = 'HOMES空き家活用事例'

#認証設定
credentials = Credentials.from_service_account_file(SP_CREDENTIAL_FILE, scopes=SP_COPE)
gc = gspread.authorize(credentials)

#ブックの選択
workbook = gc.open_by_key(SP_SHEET_KEY)

#シートの選択
worksheet = workbook.worksheet(SP_SHEET)

#スプシに入っている既存のデータをdataに代入->データフレームdf_oldに代入
data = worksheet.get_all_values()
df_old = pd.DataFrame(data[1:], columns=data[0])

#スクレイピングで取得したデータをdf_newに代入
df_new = df

#既存データと新取得データを結合
df_upload = pd.concat([df_old,df_new])
df_upload.reset_index(drop=True,inplace=True)

#スプシにデータ書き込み
set_with_dataframe(workbook.worksheet(SP_SHEET), df_upload, include_index=False)



In [85]:
themeLabel_from_html = soup.find_all('ul', attrs={'class':'themeLabel'})[20]
themeLabel_from_html.text.replace('\n            \n\n              ','/').replace('\n\n              ','').replace('\n            \n','')

'所有者の活用意識が低い/活用策のノウハウ不足/ノウハウを持つ人材の不足'

In [91]:
themeLabel_from_html = soup.find_all('div', attrs={'class':'inner'})

In [92]:
print(themeLabel_from_html)

[<div class="inner">
<dl class="descriptionList">
<dt>事業概要</dt>
<dd><div>介護施設入所時に空き家の管理責任者指名の啓発
管理責任者と専門家と新潟市が一体となった空き家の活用検討の仕組み造り
空き家活用検討に関する業界団体内の仲介委託に関するルール造り
中間所有組織を介して空き家の隣地集約による市場性の回復検証
行政経営モデルを示し、過剰な行政投資とならない公共的空き家対策
前面6ｍ道路の確保により、住宅市場へ流通性の回復と空き家再発防止</div></dd>
<dt>取り組み手法</dt>
<dd>
              空き家利活用手法の構築
            </dd>
</dl>
</div>, <div class="inner">
<ul class="themeLabel">
<li>
              活用を担う人材不足
            </li>
<li>
              ノウハウを持つ人材の不足
            </li>
<li>
              組織体制が十分でない
            </li>
<li>
              法令等により活用が困難
            </li>
</ul>
</div>, <div class="inner mod-initiativesBox">
<div>
<dl class="resultTable">
<dt>ターゲット</dt>
<dd>
                空き家の所有者、将来的な空き家の所有者、地域住民、事業者
              </dd>
<dt>事業主体</dt>
<dd>
                ＮＰＯ法人
              </dd>
<dt>提携先</dt>
<dd>
                自治体、宅建業者、行政書士、業者団体
              </dd>
</dl>
</div>
<div class="image">
<a data-lightbox="detail-image349" href="https://akiyabank-image.akiyabank.home

In [89]:
len(bukkenbox_from_html)

30

In [97]:
bukkenbox2_from_html = soup.find('div', attrs={'class':'mod-result-bukkenBox'})
inner_from_html = bukkenbox2_from_html.find_all('div', class_='inner')

inner_from_html

[<div class="inner">
 <dl class="descriptionList">
 <dt>事業概要</dt>
 <dd><div>介護施設入所時に空き家の管理責任者指名の啓発
 管理責任者と専門家と新潟市が一体となった空き家の活用検討の仕組み造り
 空き家活用検討に関する業界団体内の仲介委託に関するルール造り
 中間所有組織を介して空き家の隣地集約による市場性の回復検証
 行政経営モデルを示し、過剰な行政投資とならない公共的空き家対策
 前面6ｍ道路の確保により、住宅市場へ流通性の回復と空き家再発防止</div></dd>
 <dt>取り組み手法</dt>
 <dd>
               空き家利活用手法の構築
             </dd>
 </dl>
 </div>,
 <div class="inner">
 <ul class="themeLabel">
 <li>
               活用を担う人材不足
             </li>
 <li>
               ノウハウを持つ人材の不足
             </li>
 <li>
               組織体制が十分でない
             </li>
 <li>
               法令等により活用が困難
             </li>
 </ul>
 </div>,
 <div class="inner mod-initiativesBox">
 <div>
 <dl class="resultTable">
 <dt>ターゲット</dt>
 <dd>
                 空き家の所有者、将来的な空き家の所有者、地域住民、事業者
               </dd>
 <dt>事業主体</dt>
 <dd>
                 ＮＰＯ法人
               </dd>
 <dt>提携先</dt>
 <dd>
                 自治体、宅建業者、行政書士、業者団体
               </dd>
 </dl>
 </div>
 <div class="image">
 <a data-lightbox="detail-image